In [11]:
import gmql as gl
import random

In [3]:
sc = gl.sc

In [ ]:
l = []

for i in range(5000):
    for j in range(random.randint(0,10000)):
        a = random.randint(0,1000)
        b = random.randint(0,1000)
        c = random.randint(0,1000)
        l.append((i, (a,b,c)))

In [21]:
rdd = sc.parallelize(l)

In [23]:
mapping = {
    "pValue": 0,
    "qValue": 1,
    "start": 2
}

In [25]:
rdd.filter(lambda row: row[1][0] < 15)

In [32]:
reduced = rdd.reduceByKey(lambda x,y : (x[0] + y[0], x[1] + y[1], x[2] + y[2]))

[(0, (77295, 71490, 83206)),
 (512, (7981, 6792, 7133)),
 (4, (56241, 58069, 60843)),
 (520, (9720, 11431, 11805)),
 (172, (370616, 372945, 363861)),
 (12, (297510, 270663, 300102)),
 (16, (129417, 122151, 123632)),
 (88, (322886, 318602, 324206)),
 (904, (197442, 191901, 190555)),
 (20, (163756, 160756, 168588)),
 (840, (84656, 78141, 84819)),
 (24, (63765, 66478, 60080)),
 (516, (174305, 161891, 152970)),
 (28, (307950, 295221, 286325)),
 (768, (386498, 392575, 400323)),
 (32, (250437, 242217, 236833)),
 (688, (64998, 63767, 69465)),
 (548, (21576, 20830, 20371)),
 (40, (332734, 328149, 331762)),
 (604, (342892, 328833, 337042)),
 (956, (163621, 170036, 163534)),
 (44, (60455, 63012, 64918)),
 (48, (323003, 311665, 316190)),
 (8, (59259, 65347, 69019)),
 (564, (36853, 36844, 37195)),
 (820, (270179, 271775, 272314)),
 (56, (455032, 457936, 473429)),
 (180, (276927, 272565, 275606)),
 (572, (157521, 155582, 158963)),
 (888, (325248, 307327, 310930)),
 (64, (19738, 17637, 21527)),
 (60